## 1. Big picture

Determine likelihood of driver filing a claim.

## 2. Get the data

### Imports

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import kaggle_analysis_library as kal
%matplotlib inline

### Read Files and Examine Heads

In [2]:
sample_submission = pd.read_csv('../data/sample_submission.csv')

train = pd.read_csv('../data/train.csv')

test = pd.read_csv('../data/test.csv')

In [3]:
sample_submission.head(1)

,id,target
0,0,0.0364


In [4]:
test.head(1)

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1


In [5]:
train.head(1)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1


### Compare columns

In [6]:
train.columns

Index(['id', 'target', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15',
       'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01',
       'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat',
       'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14',
       'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin'],


In [7]:
kal.compare_columns(train,test)

Columns in df1 but not in df2 ['target']
Columns in df2 but not in df1 []


(['target'], [])

### Create prepared copy (df) to work with

In [8]:
df = train.copy()

## 3. Missing values

From description: "Values of -1 indicate that the feature was missing from the observation."

In [9]:
df = df.replace(to_replace=-1,value=np.nan)

test = test.replace(to_replace=-1,value=np.nan)

### a. Find missing values

In [10]:
test.isnull().sum()[test.isnull().sum()>0].sort_values(ascending=False)

ps_car_03_cat    616911
ps_car_05_cat    400359
ps_reg_03        161684
ps_car_14         63805
ps_car_07_cat     17331
ps_ind_05_cat      8710
ps_car_09_cat       877
ps_ind_02_cat       307
ps_car_01_cat       160
ps_ind_04_cat       145
ps_car_02_cat         5
ps_car_11             1
dtype: int64

In [11]:
#Check for missing values
df.isnull().sum()[df.isnull().sum()>0].sort_values(ascending=False)

ps_car_03_cat    411231
ps_car_05_cat    266551
ps_reg_03        107772
ps_car_14         42620
ps_car_07_cat     11489
ps_ind_05_cat      5809
ps_car_09_cat       569
ps_ind_02_cat       216
ps_car_01_cat       107
ps_ind_04_cat        83
ps_car_11             5
ps_car_02_cat         5
ps_car_12             1
dtype: int64

### b. Drop columns with large amounts of missing data:
Drop columns with 20%+ missing data

In [12]:
#Columns whereGreater than 20% missing
df.isnull().sum()[df.isnull().sum()>len(df)*.2]

ps_car_03_cat    411231
ps_car_05_cat    266551
dtype: int64

In [13]:
df.drop(['ps_car_03_cat','ps_car_05_cat'],axis=1,inplace=True)
test.drop(['ps_car_03_cat','ps_car_05_cat'],axis=1,inplace=True)

### c. Numeric Imputation

#### Solution for all numeric columns: impute median values

In [14]:
df = df.fillna(df.median())
test = test.fillna(df.median())

### d. Non-numeric Imputation

#### Find columns with non-numeric features

In [15]:
#All data types are numeric
set(df.dtypes)

{dtype('int64'), dtype('float64')}

## 4. Visualize

No visualizations at this time.

## 5. Engineer features

No features created at this time.

## 6. Seperate and Scale Data

#### Seperate ids, features, and labels

In [16]:
train_ids = df.iloc[:,0]
train_labels = df.iloc[:,1]
train_features = df.iloc[:,2:]

In [17]:
test_ids = test.iloc[:,0]
test_features = test.iloc[:,1:]

#### Scale data

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train_features)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [19]:
train_features = pd.DataFrame(scaler.transform(train_features), columns=train_features.columns)
test_features = pd.DataFrame(scaler.transform(test_features), columns=test_features.columns)

## 7. Save prepared data

In [38]:
#Train data
train_ids.to_csv('../data/train_ids.csv')
train_labels.to_csv('../data/train_labels.csv')
train_features.to_csv('../data/train_features.csv')

In [39]:
#Test data
test_ids.to_csv('../data/test_ids.csv')
test_features.to_csv('../data/test_features.csv')